<a href="https://colab.research.google.com/github/birdx0810/LearningTensorFlow/blob/main/TensorFlow___From_1_To_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow: From 1.x to 2.x

This is notebook is basically an implementation of a basic multi-layer perceptron using TF1.0 and TF2.0 method for MNIST dataset classification. The results should be similar.

In [1]:
import time

# Import MINST data 
from keras.datasets import mnist
from tqdm import trange

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelBinarizer

%matplotlib inline

In [2]:
# Set Model Parameters
learning_rate = 1e-5 
epochs = 10
batch_size = 32
num_labels = 10

In [3]:
# Set random seed
tf.random.set_seed(42)

In [4]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

train_x = train_x.reshape(train_x.shape[0], train_x.shape[1]*train_x.shape[2])
test_x = test_x.reshape(test_x.shape[0], test_x.shape[1]*test_x.shape[2])

label_binarizer = LabelBinarizer()
label_binarizer.fit(range(num_labels))

train_y = label_binarizer.transform(train_y)
test_y = label_binarizer.transform(test_y)


In [5]:
print(train_x.shape)

(60000, 784)


In [6]:
def get_batch(x, y, iteration, batch_size):
  
  start = iteration * batch_size
  end = start + batch_size

  x_mb = x[start:end]
  y_mb = y[start:end]
  
  return x_mb, y_mb

## TensorFlow 1.0


In [7]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# tf Graph Input 
x = tf.placeholder("float", [None, 784])  # mnist data image of shape 28*28 = 784 
y = tf.placeholder("float", [None, 10])   # 0-9 digits recognition => 10 classes

# Set model weight and bias 
W = tf.Variable(tf.ones([784, 10]))      # 784 -> 10 
b = tf.Variable(tf.zeros([10]))

logits = tf.matmul(x, W) + b
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))

pred = tf.nn.softmax(logits)
acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(y,1), predictions=tf.argmax(pred,1))

# Optimize model using gradient descent
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
# Initializing the variables 
init_global = tf.global_variables_initializer()
init_local = tf.local_variables_initializer()

# Launch the graph 
with tf.Session() as sess:
  sess.run(init_global)
  sess.run(init_local)
  
  # Training cycle
  for epoch in range(epochs):
    total_loss = 0.
    iterations = int(len(train_x)/batch_size)
    c = 0
    output = []
    # Loop over all batches
    for i in trange(iterations):
      # Get mini batch
      batch_xs, batch_ys = get_batch(train_x, train_y, i, batch_size)
      
      # Fit training using batch data 
      _, batch_loss = sess.run([optimizer, loss], feed_dict={x: batch_xs, y: batch_ys})

      total_loss += batch_loss/iterations

    # Display logs per epoch step
    total_loss = total_loss
    print(f"\nEpoch:{epoch+1}\ttotal loss={total_loss}")
    time.sleep(.25)

  _, accuracy = sess.run([acc, acc_op], feed_dict={x: test_x, y: test_y})
  print(f"model acc: {accuracy}")

100%|██████████| 1875/1875 [00:01<00:00, 1129.13it/s]



Epoch:1	total loss=0.3842634023229276


100%|██████████| 1875/1875 [00:01<00:00, 1193.10it/s]



Epoch:2	total loss=0.29923564769526273


100%|██████████| 1875/1875 [00:01<00:00, 1173.28it/s]


...


Epoch:8	total loss=0.2695258501455186


100%|██████████| 1875/1875 [00:01<00:00, 1208.58it/s]



Epoch:9	total loss=0.26775032064914717


100%|██████████| 1875/1875 [00:01<00:00, 1209.43it/s]



Epoch:10	total loss=0.26621794080386546
model acc: 0.9174000024795532


## TensorFlow 2.0

In [9]:
import tensorflow as tf

# Import relevant packages
from tensorflow.keras import Sequential

# First define the model 
model = Sequential()

# Add the dense layer to the model
model.add(tf.keras.layers.Dense(
  units=num_labels, 
  activation="softmax", 
  input_shape=(784,), 
  weights=[np.ones([784, 10]), np.zeros(10)]
))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Define loss and optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
criterion = tf.keras.losses.CategoricalCrossentropy()

model.compile(optimizer=optimizer, metrics=["accuracy"], loss=criterion)

In [11]:
train_history = model.fit(
  train_x, train_y, 
  batch_size=batch_size, epochs=epochs
)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.3776 - acc: 0.8988
Epoch 2/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.2908 - acc: 0.9189
Epoch 3/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.2815 - acc: 0.9211
Epoch 4/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.2737 - acc: 0.9234
Epoch 5/10
60000/60000 [==============================] - 2s 39us/sample - loss: 0.2690 - acc: 0.9258
Epoch 6/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.2666 - acc: 0.9264
Epoch 7/10
60000/60000 [==============================] - 2s 38us/sample - loss: 0.2627 - acc: 0.9273
Epoch 8/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.2610 - acc: 0.9280
Epoch 9/10
60000/60000 [==============================] - 2s 37us/sample - loss: 0.2601 - acc: 0.9279
Epoch 10/10
60000/60000 [==============================] - 

In [12]:
loss, accuracy = model.evaluate(test_x, test_y)
print(f"Acc: {accuracy}")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Acc: 0.9253000020980835


In [13]:
# get_output = tf.keras.backend.function([model.layers[0].input],[model.layers[0].output])
# layer_output = get_output(train_x)

# print(*layer_output, sep="\n")